## Домашнее задание

1. Попробуйте видоизменить параметры разобранной на уроке двухслойной нейронной сети таким образом, чтобы улучшить ее точность (число нейронов, число эпох , можно изменять число слоев).
2. Проведите анализ — что приводит к ухудшению точности нейронной сети? Что приводит к увеличению ее точности?

---

In [ ]:
# Загрузка библиотек

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

from sklearn.model_selection import train_test_split

---

In [ ]:
### Шаг 1. Определение функций, которые понадобяться для обучения
# преобразование массива в бинарный вид результатов

def to_one_hot(Y):
    n_col = np.amax(Y) + 1
    binarized = np.zeros((len(Y), n_col))
    for i in range(len(Y)):
        binarized[i, Y[i]] = 1.
    return binarized

# преобразование массива в необходимый вид

def from_one_hot(Y):
    arr = np.zeros((len(Y), 1))

    for i in range(len(Y)):
        l = Y[i]
        for j in range(len(l)):
            if(l[j] == 1):
                arr[i] = j+1
    return arr

# сигмоида и ее производная

def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_deriv(x):
    return (x)*(1 - (x))

# нормализация массива

def normalize(X, axis = -1, order = 2):
    l2 = np.atleast_1d(np.linalg.norm(X, order, axis))
    l2[l2 == 0] = 1
    return X / np.expand_dims(l2, axis)

In [ ]:
### Шаг 2. Подготовка тренировочных данных

from google.colab import drive
drive.mount('/content/drive')

# получения данных из csv файла. укажите здесь путь к файлу Iris.csv

with open('/content/drive/My Drive/Colab Notebooks/Iris.csv', 'r') as f:
    iris_data = pd.read_csv(f)

# замена текстовых значений на цифровые

iris_data['Species'].replace(['Iris-setosa', 'Iris-virginica', 'Iris-versicolor'], [0, 1, 2], inplace=True)

# формирование входных данных

columns = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
x = pd.DataFrame(iris_data, columns = columns)
x = normalize(x.values)

# формирование выходных данных(результатов)

columns = ['Species']
y = pd.DataFrame(iris_data, columns=columns)
y = y.values
y = y.flatten()
y = to_one_hot(y)

# Разделение данных на тренировочные и тестовые

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.33)


In [ ]:
### Шаг 3. Обученние нейронной сети

accuracies = { }

# определим число нейронов скрытого слоя

neurons = [5, 10, 15, 25]

for neuron_numb in neurons:
    
    # присваевание случайных весов
    
    w0 = 2*np.random.random((4, neuron_numb)) - 1 # для входного слоя   - 4 входа, 3 выхода
    w1 = 2*np.random.random((neuron_numb, 3)) - 1 # для внутреннего слоя - 5 входов, 3 выхода

    # скорость обучения (learning rate)
    
    speed = [0.05, 0.1, 0.15]
    
    for n in speed:
        
        # массив для ошибок
        
        errors = []

        epoch = [3000, 4000, 5000]
        
        # процесс обучения
        
        for epo in epoch:
            
            for i in range(epo):

                # прямое распространение(feed forward)
                
                layer0 = X_train
                layer1 = sigmoid(np.dot(layer0, w0))
                layer2 = sigmoid(np.dot(layer1, w1))

                # обратное распространение(back propagation) с использованием градиентного спуска
                
                layer2_error = y_train - layer2 # производная функции потерь = производная квадратичных потерь 
                layer2_delta = layer2_error * sigmoid_deriv(layer2)
    
                layer1_error = layer2_delta.dot(w1.T)
                layer1_delta = layer1_error * sigmoid_deriv(layer1)
            
                # коррекция
                
                w1 += layer1.T.dot(layer2_delta) * n
                w0 += layer0.T.dot(layer1_delta) * n
                
                # метрика модели
                
                error = np.mean(np.abs(layer2_error))
                errors.append(error)
                accuracy = (1 - error) * 100
            
            ### Шаг 4. Демонстрация полученных результатов
        
            layer0_t = X_test
            layer1_t = sigmoid(np.dot(layer0_t, w0))
            layer2_t = sigmoid(np.dot(layer1_t, w1))
            layer2_error_t = y_test - layer2_t

            error_t = np.mean(np.abs(layer2_error_t))
            accuracy_t = (1 - error_t) * 100
            accuracies[np.round(accuracy_t,2)] = (neuron_numb, n, epo, np.round(accuracy,2))

In [ ]:
from pprint import pprint
pprint(accuracies)

In [ ]:
best = max(accuracies)
print('Лучшее качество на тесте: {}\nКоличество нейронов: {}\nСкорость обучения: \
{}\nКоличество эпох: {}\nКачество на обучающей выборке: {}'\
.format(best, accuracies[best][0], accuracies[best][1], accuracies[best][2], accuracies[best][3]))

---

**Результат:** с высоких значений epoch и speed происходит быстрое переобучение с высоким результатом на обучающей выборке и низким на тестовой. При плавном наращивании повышается возможность следить за переобучением и контролировать финальный результат.